# ANN REGRESSION - {"BLACK FRIDAY SALES" DATASET} Gradio App

## 1. Importing Modules

In [1]:
import numpy as np
import pandas as pd
import random

from pickle import dump, load

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder

from sklearn.pipeline import Pipeline

import tensorflow
from tensorflow import keras
from tensorflow.keras import Sequential
from keras.layers import Dense, Dropout

import gradio as gr

import warnings
warnings.filterwarnings('ignore')

## 2. Loading Data and Model

### 2.1 Loading Xtrain and Model

In [2]:
X = pd.read_pickle('bfs_train_pp.pkl')
X.shape

(549868, 12)

In [3]:
def rem_cols(ds):
    cols = ['User_ID','Product_ID']
    ds = ds.drop(columns=cols)
    
    return ds

In [4]:
pipe = load(open('bfs_pipe_FE.pkl','rb'))                             # PreProcessing Pipe

mdl = tensorflow.keras.models.load_model('mdl_final.keras')           # Prediction Model

### 2.2 Loading Test Data

In [5]:
te = pd.read_pickle('bfs_test_pp.pkl')

print(f'Shape of the test dataset : {te.shape}')

Shape of the test dataset : (100, 12)


In [6]:
te.head()

,User_ID,Product_ID,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3,Purchase
264361,1004682,P00362042,M,26-35,7,B,3,1,5,7,14,8880
177205,1003441,P00157642,F,26-35,14,A,2,0,1,14,16,15173
542026,1005463,P00085442,M,36-45,0,B,3,1,12,14,17,1433
481858,1002143,P00110742,M,18-25,4,B,2,0,1,2,8,19551
226813,1004952,P00091142,F,26-35,14,A,1,1,8,14,17,8069


### 2.3 Preparing App Samples from Test Data

In [7]:
Xte = te.drop(columns='Purchase')
yte = te['Purchase']

samp = Xte.sample(10).index.values.tolist()
samp

print('App Samples with Labels ...')
te_df = pd.concat([Xte.loc[samp],yte.loc[samp]],axis=1).head(10)

App Samples with Labels ...


In [8]:
te_df

,User_ID,Product_ID,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3,Purchase
548277,1003484,P00372445,M,18-25,4,C,1,0,20,8,16,605
91068,1001985,P00121342,M,46-50,12,A,2,1,1,5,18,15562
165774,1001611,P00046742,M,26-35,0,B,1,0,1,2,15,11744
460775,1004933,P00127642,M,26-35,15,B,3,0,1,2,15,15503
409579,1003067,P00268742,F,26-35,0,A,1,0,1,8,15,15872
96377,1002929,P00148642,M,18-25,2,A,4+,0,6,10,13,19998
91440,1002038,P00200242,F,46-50,1,B,3,1,8,10,16,6145
187733,1004979,P00101042,M,36-45,2,B,1,1,5,12,14,5202
511023,1000752,P00016342,F,26-35,3,A,3,0,1,2,8,4012
102186,1003778,P00295342,M,26-35,0,B,4+,1,2,8,15,13296


In [9]:
print('App Samples without Labels ...')
gr_samples = Xte.loc[samp].values.tolist()

App Samples without Labels ...


In [10]:
gr_samples

[[1003484, 'P00372445', 'M', '18-25', 4, 'C', '1', 0, 20, 8, 16],
 [1001985, 'P00121342', 'M', '46-50', 12, 'A', '2', 1, 1, 5, 18],
 [1001611, 'P00046742', 'M', '26-35', 0, 'B', '1', 0, 1, 2, 15],
 [1004933, 'P00127642', 'M', '26-35', 15, 'B', '3', 0, 1, 2, 15],
 [1003067, 'P00268742', 'F', '26-35', 0, 'A', '1', 0, 1, 8, 15],
 [1002929, 'P00148642', 'M', '18-25', 2, 'A', '4+', 0, 6, 10, 13],
 [1002038, 'P00200242', 'F', '46-50', 1, 'B', '3', 1, 8, 10, 16],
 [1004979, 'P00101042', 'M', '36-45', 2, 'B', '1', 1, 5, 12, 14],
 [1000752, 'P00016342', 'F', '26-35', 3, 'A', '3', 0, 1, 2, 8],
 [1003778, 'P00295342', 'M', '26-35', 0, 'B', '4+', 1, 2, 8, 15]]

## 3. Gradio App Development

### 3.1 Gradio App Prediction Function

In [11]:
def bfs_prediction(uid, pid, gen, age, occ, cc, stay, ms, pc1, pc2, pc3):
               
    cols = ['User_ID','Product_ID','Gender','Age','Occupation','City_Category','Stay_In_Current_City_Years', 
            'Marital_Status', 'Product_Category_1', 'Product_Category_2', 'Product_Category_3']
    
    data = np.array([uid, pid, gen, age, occ, cc, stay, ms, pc1, pc2, pc3], dtype=object).reshape(1,11)      # input
    data = pd.DataFrame(data=data, columns=cols, dtype='object')                                             # data
      
    pp = pipe.transform(data)                                                              # data preprocessing and FE
    
    res = mdl.predict(pp, verbose=0)[0][0]                                                 # prediction
    
    return f'Predicted Purchase Amount: $ {res:0.2f}'

In [12]:
res = bfs_prediction(1002929, 'P00148642', 'M', '18-25', 2, 'A', '4+', 0, 6, 10, 13)
print(res)

Predicted Purchase Amount: $ 16019.96


### 3.2 Gradio App Interface for Input and Output

In [13]:
iface = gr.Interface(bfs_prediction, 
        inputs=[
                gr.Textbox(default=None, label='User ID'), 
                gr.Textbox(default=None, label='Product ID'), 
                gr.Dropdown(X[X.columns.tolist()[0]].unique().tolist(), value=X[X.columns.tolist()[0]].unique().tolist()[0], label='Gender'),
                gr.Dropdown(X[X.columns.tolist()[1]].unique().tolist(), value=X[X.columns.tolist()[1]].unique().tolist()[0], label='Age'),
                gr.Dropdown(X[X.columns.tolist()[2]].unique().tolist(), value=X[X.columns.tolist()[2]].unique().tolist(), label='Occupation'),
                gr.Dropdown(X[X.columns.tolist()[3]].unique().tolist(), value=X[X.columns.tolist()[3]].unique().tolist()[0], label='City_Category'),
                gr.Dropdown(X[X.columns.tolist()[4]].unique().tolist(), value=X[X.columns.tolist()[4]].unique().tolist()[0], label='Stay_In_Current_City_Years'),
                gr.Dropdown(X[X.columns.tolist()[5]].unique().tolist(), value=X[X.columns.tolist()[5]].unique().tolist()[0], label='Marital_Status'),
                gr.Dropdown(X[X.columns.tolist()[6]].unique().tolist(), value=X[X.columns.tolist()[6]].unique().tolist(), label='Product_Category_1'),
                gr.Dropdown(X[X.columns.tolist()[7]].unique().tolist(), value=X[X.columns.tolist()[7]].unique().tolist(), label='Product_Category_2'),
                gr.Dropdown(X[X.columns.tolist()[8]].unique().tolist(), value=X[X.columns.tolist()[8]].unique().tolist(), label='Product_Category_3') 
               ], 
        outputs="text",
        layout='horizontal', theme='soft', 
        title='Black Friday Sales Predictor App', 
        description='Provide Details of the User and Product', 
        article='Created by: Manish Sharma',
        examples=gr_samples) 

### 3.3 Gradio App Launch

In [14]:
iface.launch(share=False, debug=True, width=1125, height=750)

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


Keyboard interruption in main thread... closing server.
